<a href="https://colab.research.google.com/github/HenryMakerCH/so-vits-svc-inference/blob/main/sovits4_0%E4%B8%80%E9%94%AE%E8%AE%AD%E7%BB%83_%E6%8E%A8%E7%90%86%E8%84%9A%E6%9C%AC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# 查看显卡
!nvidia-smi

Mon Feb 20 05:45:26 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 510.47.03    Driver Version: 510.47.03    CUDA Version: 11.6     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   66C    P0    32W /  70W |      0MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
#@title 克隆的github仓库
!git clone https://github.com/innnky/so-vits-svc -b 4.0


Cloning into 'so-vits-svc'...
remote: Enumerating objects: 728, done.
remote: Counting objects: 100% (728/728), done.
remote: Compressing objects: 100% (363/363), done.
remote: Total 728 (delta 359), reused 689 (delta 337), pack-reused 0
Receiving objects: 100% (728/728), 25.27 MiB | 16.41 MiB/s, done.
Resolving deltas: 100% (359/359), done.


In [ ]:
#@title 安装依赖
%cd /content/so-vits-svc
!pip install pyworld praat-parselmouth fairseq


/content/so-vits-svc
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.0/11.0 MB 81.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 123.8/123.8 KB 16.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 269.5/269.5 KB 30.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 118.9/118.9 KB 12.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 112.4/112.4 KB 13.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for antlr4-python3-runtime: filename=antlr4_python3_runtime-4.8-py3-none-any.whl size=141231 sha256=1de647a9a825340b05e474f135e12ff0f073288edc3eb369e9a1b3139f62c7a1
  Stored in directory: /root/.cache/pip/wheels/c8/d0/ab/d43c02eaddc5b9004db86950802442ad9a26f279c619e28da0
Successfully built antlr4-python3-runtime


In [ ]:
#@title 下载必要模型文件
# 源仓库地址：[contentvec](https://github.com/auspicious3000/contentvec)
# 模型原下载链接：[checkpoint_best_legacy_500.pt](https://ibm.box.com/s/z1wgl1stco8ffooyatzdwsqn2psd9lrr)
# 由于源网盘无法提供http直链，根据mit协议，对模型进行二次分发，提供下载直链
!wget -P hubert/ https://huggingface.co/innnky/contentvec/resolve/main/checkpoint_best_legacy_500.pt

--2023-02-20 05:57:07--  https://huggingface.co/innnky/contentvec/resolve/main/checkpoint_best_legacy_500.pt
Resolving huggingface.co (huggingface.co)... 54.235.118.239, 3.231.67.228, 2600:1f18:147f:e800:671:b733:ecf3:a585, ...
Connecting to huggingface.co (huggingface.co)|54.235.118.239|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://cdn-lfs.huggingface.co/repos/0a/5f/0a5f89ff9058782f4b7a207d38e0914d63f6f55e8fc1fad0c3bf68b546039f2d/60d936ec5a566776fc392e69ad8b630d14eb588111233fe313436e200a7b187b?response-content-disposition=attachment%3B+filename*%3DUTF-8%27%27checkpoint_best_legacy_500.pt%3B+filename%3D%22checkpoint_best_legacy_500.pt%22%3B&Expires=1677131827&Policy=eyJTdGF0ZW1lbnQiOlt7IlJlc291cmNlIjoiaHR0cHM6Ly9jZG4tbGZzLmh1Z2dpbmdmYWNlLmNvL3JlcG9zLzBhLzVmLzBhNWY4OWZmOTA1ODc4MmY0YjdhMjA3ZDM4ZTA5MTRkNjNmNmY1NWU4ZmMxZmFkMGMzYmY2OGI1NDYwMzlmMmQvNjBkOTM2ZWM1YTU2Njc3NmZjMzkyZTY5YWQ4YjYzMGQxNGViNTg4MTExMjMzZmUzMTM0MzZlMjAwYTdiMTg3Yj9yZXNwb25zZS1jb250

# 数据集预处理

In [ ]:
#@title 加载Google云端硬盘
#@markdown 加载Google云端硬盘
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


该处理脚本可以一次性预处理多个说话人，并且生成多说话人filelists以及对应的配置文件

只需要将你的数据集按照以下文件结构放到dataset_raw目录下


```
dataset_raw
├───speaker0
│   ├───xxx1-xxx1.wav
│   ├───...
│   └───Lxx-0xx8.wav
└───speaker1
    ├───xx2-0xxx2.wav
    ├───...
    └───xxx7-xxx007.wav

```




In [ ]:
#@title 从谷歌云盘加载打包好的数据集进行预处理
#@markdown **sovits3.0的数据集不再需要特定的文件结构，将数据集的所有wav文件放在同一个文件夹下压缩为zip后上传至谷歌云盘即可，该处理脚本可以一次性预处理多个数据集，处理多个数据集时请依次解压每一个数据集**

#@markdown 数据集名称（**人物的英文/拼音名**，与建数据文件夹时统一；不带zip。）
DATASETNAME = "kiritan"  #@param {type:"string"}
#@markdown 压缩包路径（谷歌盘路径，传到dataset的就不改这个，没有dataset文件夹就新建一个）
ZIP_PATH = "/content/drive/MyDrive/dataset/"  #@param {type:"string"}
ZIP_NAME = ZIP_PATH + DATASETNAME

!unzip -d /content/so-vits-svc/dataset_raw {ZIP_NAME}.zip

Archive:  /content/drive/MyDrive/dataset/kiritan.zip
   creating: /content/so-vits-svc/dataset_raw/kiritan/
  inflating: /content/so-vits-svc/dataset_raw/kiritan/000047.wav  
  inflating: /content/so-vits-svc/dataset_raw/kiritan/000046.wav  
  inflating: /content/so-vits-svc/dataset_raw/kiritan/000050.wav  
  inflating: /content/so-vits-svc/dataset_raw/kiritan/000044.wav  
  inflating: /content/so-vits-svc/dataset_raw/kiritan/000045.wav  
  inflating: /content/so-vits-svc/dataset_raw/kiritan/000041.wav  
  inflating: /content/so-vits-svc/dataset_raw/kiritan/000040.wav  
  inflating: /content/so-vits-svc/dataset_raw/kiritan/000042.wav  
  inflating: /content/so-vits-svc/dataset_raw/kiritan/000043.wav  
  inflating: /content/so-vits-svc/dataset_raw/kiritan/000018.wav  
  inflating: /content/so-vits-svc/dataset_raw/kiritan/000024.wav  
  inflating: /content/so-vits-svc/dataset_raw/kiritan/000030.wav  
  inflating: /content/so-vits-svc/dataset_raw/kiritan/000031.wav  
  inflating: /content

In [ ]:
#@title 重采样到44100hz
!python resample.py

./dataset_raw/kiritan
51it [00:00, 89.46it/s]


In [ ]:
#@title 划分训练集 生成配置文件
!python preprocess_flist_config.py

100% 1/1 [00:00<00:00, 4359.98it/s]
Writing ./filelists/train.txt
100% 47/47 [00:00<00:00, 791695.94it/s]
Writing ./filelists/val.txt
100% 2/2 [00:00<00:00, 74898.29it/s]
Writing ./filelists/test.txt
100% 2/2 [00:00<00:00, 77672.30it/s]
Writing configs/config.json


In [ ]:
#@title 生成hubert和f0
!python preprocess_hubert_f0.py

[51]
Loading hubert for content...
load model(s) from hubert/checkpoint_best_legacy_500.pt
INFO:fairseq.tasks.text_to_speech:Please install tensorboardX: pip install tensorboardX
INFO:fairseq.tasks.hubert_pretraining:current directory is /content/so-vits-svc
INFO:fairseq.tasks.hubert_pretraining:HubertPretrainingTask Config {'_name': 'hubert_pretraining', 'data': 'metadata', 'fine_tuning': False, 'labels': ['km'], 'label_dir': 'label', 'label_rate': 50.0, 'sample_rate': 16000, 'normalize': False, 'enable_padding': False, 'max_keep_size': None, 'max_sample_size': 250000, 'min_sample_size': 32000, 'single_target': False, 'random_crop': True, 'pad_audio': False}
INFO:fairseq.models.hubert.hubert:HubertModel Config: {'_name': 'hubert', 'label_rate': 50.0, 'extractor_mode': default, 'encoder_layers': 12, 'encoder_embed_dim': 768, 'encoder_ffn_embed_dim': 3072, 'encoder_attention_heads': 12, 'activation_fn': gelu, 'layer_type': transformer, 'dropout': 0.1, 'attention_dropout': 0.1, 'activati

In [ ]:
#@title 至此 数据集预处理制作完毕，将数据集和相关文件保存到谷歌云盘的dataset文件夹下，方便下次训练使用
#压缩dataset文件夹
!zip -r dataset.zip /content/so-vits-svc/dataset
#@markdown 自定义备份到谷歌云盘的dataset文件夹下的数据集文件夹名，避免混淆
dataset_name_drive = "kiritan_preprocessed"  #@param {type:"string"}
DATASET_PATH_DRIVE = "/content/drive/MyDrive/dataset/" + dataset_name_drive
!mkdir -p {DATASET_PATH_DRIVE}

!cp /content/so-vits-svc/dataset.zip "{DATASET_PATH_DRIVE}"
!cp configs/config.json "{DATASET_PATH_DRIVE}"
!cp filelists/train.txt "{DATASET_PATH_DRIVE}"
!cp filelists/val.txt "{DATASET_PATH_DRIVE}"

  adding: content/so-vits-svc/dataset/ (stored 0%)
  adding: content/so-vits-svc/dataset/44k/ (stored 0%)
  adding: content/so-vits-svc/dataset/44k/kiritan/ (stored 0%)
  adding: content/so-vits-svc/dataset/44k/kiritan/000049.wav.soft.pt (deflated 8%)
  adding: content/so-vits-svc/dataset/44k/kiritan/000020.wav.soft.pt (deflated 8%)
  adding: content/so-vits-svc/dataset/44k/kiritan/000027.wav.f0.npy (deflated 46%)
  adding: content/so-vits-svc/dataset/44k/kiritan/000007.wav.f0.npy (deflated 30%)
  adding: content/so-vits-svc/dataset/44k/kiritan/000026.wav.f0.npy (deflated 21%)
  adding: content/so-vits-svc/dataset/44k/kiritan/000000.wav (deflated 8%)
  adding: content/so-vits-svc/dataset/44k/kiritan/000039.wav.soft.pt (deflated 8%)
  adding: content/so-vits-svc/dataset/44k/kiritan/000035.wav.soft.pt (deflated 8%)
  adding: content/so-vits-svc/dataset/44k/kiritan/000024.wav.soft.pt (deflated 8%)
  adding: content/so-vits-svc/dataset/44k/kiritan/000004.wav (deflated 5%)
  adding: content

In [ ]:
#@title 已经预处理过数据集的话，就可以跳过预处理部分 直接从云盘解压处理过的数据 以及配置文件
#@markdown 从谷歌云盘加载预处理过的数据集，文件夹名和你备份的时候输入的一样
back_up_name = "kiritan_preprocessed"  #@param {type:"string"}
BACK_UP_DATASET_PATH = "/content/drive/MyDrive/dataset/" + back_up_name
!unzip {BACK_UP_DATASET_PATH}/dataset.zip -d /
!cp {BACK_UP_DATASET_PATH}/config.json /content/so-vits-svc/configs/config.json 
!cp {BACK_UP_DATASET_PATH}/val.txt filelists/val.txt
!cp {BACK_UP_DATASET_PATH}/train.txt filelists/train.txt


# 拷贝云盘上保存的记录点
# !cp /content/drive/MyDrive/G_800.pth logs/48k/
# !cp /content/drive/MyDrive/D_800.pth logs/48k/

Archive:  /content/drive/MyDrive/dataset/kiritan_preprocessed/dataset.zip
replace /content/so-vits-svc/dataset/44k/kiritan/000049.wav.soft.pt? [y]es, [n]o, [A]ll, [N]one, [r]ename: A
  inflating: /content/so-vits-svc/dataset/44k/kiritan/000049.wav.soft.pt  
  inflating: /content/so-vits-svc/dataset/44k/kiritan/000020.wav.soft.pt  
  inflating: /content/so-vits-svc/dataset/44k/kiritan/000027.wav.f0.npy  
  inflating: /content/so-vits-svc/dataset/44k/kiritan/000007.wav.f0.npy  
  inflating: /content/so-vits-svc/dataset/44k/kiritan/000026.wav.f0.npy  
  inflating: /content/so-vits-svc/dataset/44k/kiritan/000000.wav  
  inflating: /content/so-vits-svc/dataset/44k/kiritan/000039.wav.soft.pt  
  inflating: /content/so-vits-svc/dataset/44k/kiritan/000035.wav.soft.pt  
  inflating: /content/so-vits-svc/dataset/44k/kiritan/000024.wav.soft.pt  
  inflating: /content/so-vits-svc/dataset/44k/kiritan/000004.wav  
  inflating: /content/so-vits-svc/dataset/44k/kiritan/000029.wav  
  inflating: /conte

# 训练

In [ ]:
#@title  选择是否将训练后的模型保存到谷歌云盘 和 是否使用预模型

#@markdown **将训练后的模型文件保存到谷歌云盘，勾选后，恢复训练时也需要勾选并执行**
Save_to_drive = True #@param {type:"boolean"}
if Save_to_drive:
  !rm -rf /content/so-vits-svc/logs/44k
  !mkdir -p /content/drive/MyDrive/44k
  !ln -s /content/drive/MyDrive/44k /content/so-vits-svc/logs/44k

#@markdown **首次训练下载预模型 之后继续训练则使用自己保存的记录点，无需再下载**

#@markdown **使用预模型，下面打钩自动下载并启用**
pre_pth = True #@param {type:"boolean"}
if pre_pth:
  !wget -P logs/44k/ https://huggingface.co/innnky/sovits_pretrained/resolve/main/sovits4/G_0.pth
  !wget -P logs/44k/ https://huggingface.co/innnky/sovits_pretrained/resolve/main/sovits4/D_0.pth


--2023-02-20 06:05:15--  https://huggingface.co/innnky/sovits_pretrained/resolve/main/sovits4/G_0.pth
Resolving huggingface.co (huggingface.co)... 3.231.67.228, 54.235.118.239, 2600:1f18:147f:e800:671:b733:ecf3:a585, ...
Connecting to huggingface.co (huggingface.co)|3.231.67.228|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://cdn-lfs.huggingface.co/repos/b8/68/b8680b5490e63bf5f140618c3ee2e5ccd0731956a87ab87a228c08398cd8e03c/20a327c54e5731bed377bd38404bc32ab98e66a1b2777b0af4cc034d4d6914b0?response-content-disposition=attachment%3B+filename*%3DUTF-8%27%27G_0.pth%3B+filename%3D%22G_0.pth%22%3B&Expires=1677132316&Policy=eyJTdGF0ZW1lbnQiOlt7IlJlc291cmNlIjoiaHR0cHM6Ly9jZG4tbGZzLmh1Z2dpbmdmYWNlLmNvL3JlcG9zL2I4LzY4L2I4NjgwYjU0OTBlNjNiZjVmMTQwNjE4YzNlZTJlNWNjZDA3MzE5NTZhODdhYjg3YTIyOGMwODM5OGNkOGUwM2MvMjBhMzI3YzU0ZTU3MzFiZWQzNzdiZDM4NDA0YmMzMmFiOThlNjZhMWIyNzc3YjBhZjRjYzAzNGQ0ZDY5MTRiMD9yZXNwb25zZS1jb250ZW50LWRpc3Bvc2l0aW9uPSoiLCJDb25kaXRpb24iOnsiRGF0ZUxlc

In [ ]:
#@title  开始训练

#@markdown **开始训练**

#@markdown **启用tensorboard可视化数据**
tensorboard_on = True #@param {type:"boolean"}
if tensorboard_on:
  %load_ext tensorboard
  %tensorboard --logdir logs/44k

!python train.py -c configs/config.json -m 44k


In [ ]:
#@title 手动将训练后的模型文件备份到谷歌云盘
#@markdown 需要自己查看/content/so-vits-svc/logs/44k/文件夹下模型的文件名，手动修改下方命令末尾的文件名
!mv /content/so-vits-svc/logs/44k/G_1000.pth /content/drive/MyDrive
!mv /content/so-vits-svc/logs/44k/D_1000.pth /content/drive/MyDrive

# 推理

In [ ]:
#@title 合成音频（推理）
#@markdown 需要将音频上传到so-vits-svc/raw 文件夹下, 然后设置模型路径、配置文件路径、合成的音频名称

!python inference_main.py -m "logs/44k/G_1600.pth" -c "configs/config.json" -n "君の知らない物語-src.wav" -s kiritan


load model(s) from hubert/checkpoint_best_legacy_500.pt
INFO:fairseq.tasks.text_to_speech:Please install tensorboardX: pip install tensorboardX
INFO:fairseq.tasks.hubert_pretraining:current directory is /content/so-vits-svc
INFO:fairseq.tasks.hubert_pretraining:HubertPretrainingTask Config {'_name': 'hubert_pretraining', 'data': 'metadata', 'fine_tuning': False, 'labels': ['km'], 'label_dir': 'label', 'label_rate': 50.0, 'sample_rate': 16000, 'normalize': False, 'enable_padding': False, 'max_keep_size': None, 'max_sample_size': 250000, 'min_sample_size': 32000, 'single_target': False, 'random_crop': True, 'pad_audio': False}
INFO:fairseq.models.hubert.hubert:HubertModel Config: {'_name': 'hubert', 'label_rate': 50.0, 'extractor_mode': default, 'encoder_layers': 12, 'encoder_embed_dim': 768, 'encoder_ffn_embed_dim': 3072, 'encoder_attention_heads': 12, 'activation_fn': gelu, 'layer_type': transformer, 'dropout': 0.1, 'attention_dropout': 0.1, 'activation_dropout': 0.0, 'encoder_layerdro